# FlowSOM Clustering

In this tutorial we are going to review how you can use FlowSOM to cluster single cell cytometry data. To demonstrate the FlowSOM algorithm we will be using the Levine 32 dimensional mass cytometry data originally introduced in the Phenograph paper "Data-Driven Phenotypic Dissection of AML Reveals Progenitor-like Cells that Correlate with Prognosis" in 2015 (https://pubmed.ncbi.nlm.nih.gov/26095251/). You can obtain the Levine data from this GitHub repository: https://github.com/lmweber/benchmark-data-Levine-32-dim/tree/master/data.

In [ ]:
import scanpy as sc
import anndata as ann
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import datetime
import pytometry as pm


sc.logging.print_versions()
sc.settings.verbosity = 3

First we will load the data.

In [ ]:
adata = pm.io.read_fcs("./Levine_32dim_notransform.fcs")

In [ ]:
adata.var

As you can see, the dataset consists of numerous channels and some meta-variables such as individual and label.

In [ ]:
adata[:, "label"].to_df()["label"].value_counts(dropna=False)

In [ ]:
adata[:, "individual"].to_df()["individual"].value_counts(dropna=False)

In [ ]:
channels = [
    "DNA1",
    "DNA2",
    "CD45RA",
    "CD133",
    "CD19",
    "CD22",
    "CD11b",
    "CD4",
    "CD8",
    "CD34",
    "Flt3",
    "CD20",
    "CXCR4",
    "CD235ab",
    "CD45",
    "CD123",
    "CD321",
    "CD14",
    "CD33",
    "CD47",
    "CD11c",
    "CD7",
    "CD15",
    "CD16",
    "CD44",
    "CD38",
    "CD13",
    "CD3",
    "CD61",
    "CD117",
    "CD49d",
    "HLA-DR",
    "CD64",
    "CD41",
]

We want to normalise the channels with a hyperbolic inverse sine with cofactor 5 (as performed in the original paper)

In [ ]:
adata_arcsinh = pm.tl.normalize_arcsinh(adata[:, channels], cofactor=5, inplace=False)

We can now cluster our data with the `flowsom_clustering` function from the `tl` (tools) module of `pytometry`. The FlowSOM algorithm works in the following stages:
1. A self-organsing map is trained that learns a low dimensional embedding of the single cell feature space
2. Consensus clustering (https://github.com/burtonrj/consensusclustering) of the SOM nodes identifies the optimal number of clusters to retain
3. Each observation is assigned to a cluster according to its nearest meta-cluster in the SOM

The function is going to take as input:
* `adata` - our annotated dataframe of normalised channels
* `key_added` - the key to add the cluster labels under
* `som_dim` - the size of the self-organising map; (10, 10) is the default size used by the original FlowSOM implementation and usually does not need to be altered
* `sigma` - this controls the radius of the neighbourhood function and again the default value of 1.0 works well in most cases
* `learning_rate` - controls the size of weight vector in learning of SOM and the default value of 0.5 works well in most cases
* `batch_size` - adjust if you have memory constraints 
* `seed` - random seed for reproducibility
* `min_clusters` - the minimum number of clusters when searching for the optimal number of clusters in consensus clustering
* `max_clusters` - the maximum number of clusters when searching for the optimal number of clusters in consensus clustering
* `return_clustering_objs` - if True, the SOM and Consensus Clustering objects are returned; this can be helpful for inspecting the results as shown later
* `verbose` - show progress bars etc
* `copy` - copy the `adata` object (recommended)
* `agglomerative_clustering_kwargs` - keyword arguments passed to the `AgglomerativeClustering` Scikit-Learn object used by the consensus clustering

In [ ]:
adata_flowsom, cluster_objs = pm.tl.flowsom_clustering(
    adata=adata_arcsinh,
    key_added="flowsom_clusters",
    som_dim=(10, 10),
    sigma=1.0,
    learning_rate=0.5,
    batch_size=500,
    seed=42,
    min_clusters=10,
    max_clusters=20,
    return_clustering_objs=True,
    verbose=True,
    copy=True,
    agglomerative_clustering_kwargs={"metric": "euclidean", "linkage": "ward"},
)

In [ ]:
adata_flowsom.obs["flowsom_clusters"].value_counts()

FlowSOM has generated 15 clusters. Lets visualise them using PCA and UMAP:

In [ ]:
sc.pp.pca(adata_flowsom)
pl.rcParams["figure.figsize"] = (5, 5)
sc.pl.pca_overview(
    adata_flowsom,
    color="flowsom_clusters",
)

In [ ]:
sc.pp.pca(adata_flowsom, n_comps=10)
sc.pp.neighbors(adata_flowsom, n_neighbors=15, use_rep="X_pca")

In [ ]:
sc.tl.umap(adata_flowsom)

In [ ]:
sc.pl.umap(adata_flowsom, color="flowsom_clusters")

We can inspect why 15 clusters were chosen by accessing the `ConsensusClustering` object returned by the our FlowSOM function. Consensus clustering works by calculating the consensus matrix of observations across random resamples of our data - stable clustering should result in observations consistently existing within the same or different clusters, if they sometimes appear in the same cluster or sometime do not, this suggests the clustering is unstable.

We can plot the CDF of the consensus matrix across different `k` clusters and observe the change point at which the area under the CDF no longer increased with the same magnitude. The plots below show the change point as detected by consensus clustering (first plot) and the CDF for each value of `k` (second plot).

In [ ]:
cluster_objs["consensus_clustering"].plot_auc_cdf()

In [ ]:
cluster_objs["consensus_clustering"].plot_cdf()

The change in CDF below and above a `k` of 15 is subtle and we would expect ideal clustering to have consensus index values occuring around 0 or 1, but biological data is often noisy and consensus clusters can be difficult to identify.

The consensus matrix for the chosen `k` of 15 is shown below. Notice how for many observations a clear consensus is established along the diagonal but some observations are more difficult and have a 'fuzzy' allocation across clusters.

In [ ]:
cluster_objs["consensus_clustering"].plot_clustermap(15, figsize=(6, 6))